<a href="https://colab.research.google.com/github/devinlin2022/Streamlit/blob/main/RISI_Report_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install pygsheets
!pip install google-colab-selenium
!pip install retrying
!pip install PyMuPDF==1.24.9
!pip install fitz==1.16.14

import base64
import io
import time
import pandas as pd
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import pymupdf
# from pdf2image import convert_from_path
# from fitz import open as fitz_open
import pygsheets
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
import gspread
import re
import json
# Authenticate user
auth.authenticate_user()
creds, _ = default()

def save_pdf(driver, path):
    settings = {
        "landscape": False,
        "displayHeaderFooter": False,
        "printBackground": True,
        "preferCSSPageSize": True
    }
    result = driver.execute_cdp_cmd("Page.printToPDF", settings)
    pdf_data = base64.b64decode(result['data'])
    with open(path, 'wb') as f:
        f.write(pdf_data)


# def save_as_jpg(driver, path):

#     screenshot = driver.get_screenshot_as_png()

#     image = Image.open(io.BytesIO(screenshot))

#     image = image.convert('RGB')
#     image.save(path, 'JPEG')

def fetch_RISI_data(link):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    download_dir = "/content"  # 换成你希望保存文件的目录
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(link)

    wait = WebDriverWait(driver, 100)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#userEmail')))

    driver.execute_script(f'document.querySelector("#userEmail").value = "devin_lin@colpal.com"')
    driver.execute_script(f'document.querySelector("#password").value = "Abcd0816,"')
    driver.execute_script(f'document.querySelector("#login-button").click()')
    time.sleep(5)

    # Dynamic search term
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#cells-container > fui-grid-cell > fui-widget > header > fui-widget-actions > div:nth-child(1) > button > span.mat-mdc-button-touch-target')))
    driver.execute_script(f'document.querySelector("#cells-container > fui-grid-cell > fui-widget > header > fui-widget-actions > div:nth-child(1) > button > span.mat-mdc-button-touch-target").click()')
    time.sleep(2)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#mat-menu-panel-3 > div > div > div:nth-child(2) > fui-export-dropdown-item:nth-child(3) > button > span")))
    # driver.execute_script(f'document.querySelector("#mat-menu-panel-3 > div > div > div:nth-child(2) > fui-export-dropdown-item:nth-child(3) > button > span").click()')
    input_element = driver.execute_script("return document.querySelector('#mat-menu-panel-3 > div > div > div:nth-child(2) > fui-export-dropdown-item:nth-child(3) > button > span')")
    try:
    # Wait up to 10 seconds for the element to be clickable
      wait = WebDriverWait(driver, 60)
      input_element = wait.until(
          EC.element_to_be_clickable((By.CSS_SELECTOR, "#mat-menu-panel-3 > div > div > div:nth-child(2) > fui-export-dropdown-item:nth-child(3) > button > span"))
      )
      input_element.click()
      print("Element clicked successfully!")

    except Exception as e:
      print(f"An error occurred: {e}")

    save_pdf(driver, '/content/webpage.pdf')
    time.sleep(10)
    driver.close()

    return driver

import os
import time
from google.colab import auth
from google.auth import default
# Authenticate user
auth.authenticate_user()
creds, _ = default()

def wait_file_and_rename(download_dir, old_ext, new_filename, timeout=30):
    time_counter = 0
    while time_counter < timeout:
        files = [f for f in os.listdir(download_dir) if f.endswith(old_ext)]
        if files:
            old_file = os.path.join(download_dir, files[0])
            new_file = os.path.join(download_dir, new_filename)
            os.rename(old_file, new_file)
            return new_file
        time.sleep(1)
        time_counter += 1
    raise Exception("Download file not found!")

def clean_palm_csv(input_path, output_path):
    df = pd.read_csv(input_path, skiprows=2)
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df = df.drop(0).reset_index(drop=True)
    df = df.drop(0).reset_index(drop=True)
    df = df.iloc[:-5]
    for column in df.columns:
      # Get the last valid (non-NaN) value in the column
      last_valid_value = df[column].dropna().iloc[-1] if not df[column].dropna().empty else None
      if last_valid_value is not None:
          df[column] = df[column].fillna(last_valid_value)
      # If the entire column was NaN, last_valid_value would be None,
      # and the column will remain all NaN (or you can choose a default fill value)

      df.to_csv(output_path, index=False)

def sync_and_dedup_csv_to_gsheet(csv_path, gsheet_id, sheet_title, service_account_json):

    df_new = pd.read_csv(csv_path)

    gc = pygsheets.client.Client(creds)
    sh = gc.open_by_key(gsheet_id)
    wks = sh.worksheet_by_title(sheet_title)

    try:
        df_old = wks.get_as_df(has_header=True, include_tailing_empty=False)
        df_all = pd.concat([df_old, df_new], ignore_index=True).dropna(how='all')
    except Exception:
        df_all = df_new

    df_all = df_all.drop_duplicates()

    wks.clear()
    wks.set_dataframe(df_all, (1,1))

    # nrows = df_all.shape[0] + 1
    # cell_range = wks.range(f'A2:A{nrows}')
    # for cell in cell_range:
    #     cell.set_number_format('DATE', 'yyyy-mm-dd')
fetch_RISI_data('https://dashboard.fastmarkets.com/sw/x2TtMTTianBBefSdGCeZXc/palm-oil-global-prices')
download_dir = "/content"
new_filename = "Palm.csv"
downloaded_file = wait_file_and_rename(download_dir, ".csv", new_filename)
print(f"Download and rename as：{downloaded_file}")
clean_palm_csv("Palm.csv","Palm_cleaned.csv")
sync_and_dedup_csv_to_gsheet(
    csv_path='Palm_cleaned.csv',
    gsheet_id='1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE',
    sheet_title='Palm Oil Price',
    service_account_json=creds
)


ERROR: Could not find a version that satisfies the requirement fitz==1.16.14 (from versions: 0.0.1.dev0, 0.0.1.dev2)
ERROR: No matching distribution found for fitz==1.16.14
Element clicked successfully!
Download and rename as：/content/Palm.csv
